# 통계적 검정 실습파일

# 순열검정 (Permutation_test)

재표본추출을 직접 해보고싶어서 만든 페이지입니다.

가진 데이터를 활용하여(이 통계 교재 실습을 위해 만들어진 데이터가 아닌) 귀무가설을 기각하는 경우를 보고싶었습니다.

이전에 AI 공부를 할 때 함께했던 멜버른 집값 데이터를 데려옵니다.

## Data, library import

In [1]:
# 라이브러리 불러오기

%matplotlib inline

from pathlib import Path
import random

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from scipy import stats
from sklearn.utils import resample
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import power

import seaborn as sns
import matplotlib.pylab as plt

import warnings

In [2]:
mel_price = pd.read_csv('/Users/Angela/Desktop/Personal/AI_data/melb_data.csv')

## data info

In [3]:
mel_price.nunique()

Suburb             314
Address          13378
Rooms                9
Type                 3
Price             2204
Method               5
SellerG            268
Date                58
Distance           202
Postcode           198
Bedroom2            12
Bathroom             9
Car                 11
Landsize          1448
BuildingArea       602
YearBuilt          144
CouncilArea         33
Lattitude         6503
Longtitude        7063
Regionname           8
Propertycount      311
dtype: int64

In [4]:
mel_price.sample(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
5747,Spotswood,10 Raleigh St,3,h,855000.0,S,RT,6/08/2016,7.7,3015.0,...,1.0,1.0,326.0,112.0,1991.0,Hobsons Bay,-37.8310,144.8884,Western Metropolitan,1223.0
4195,Moorabbin,24a Franklin St,3,t,800000.0,S,Buxton,28/08/2016,14.6,3189.0,...,2.0,1.0,0.0,NaN,1999.0,Kingston,-37.9374,145.0542,Southern Metropolitan,2555.0
6570,Windsor,6/213 Dandenong Rd,2,u,600000.0,S,hockingstuart,6/08/2016,5.1,3181.0,...,1.0,1.0,1043.0,NaN,NaN,Stonnington,-37.8581,144.9972,Southern Metropolitan,4380.0
6019,Sunshine North,2/86 Phoenix St,2,u,315000.0,PI,Douglas,8/10/2016,13.3,3020.0,...,1.0,1.0,14.0,NaN,NaN,Brimbank,-37.7756,144.8345,Western Metropolitan,4217.0
7963,Hawthorn East,6/4 Wiseman St,1,u,400500.0,S,Bekdon,13/05/2017,7.5,3123.0,...,1.0,1.0,71.0,NaN,NaN,Boroondara,-37.8372,145.0433,Southern Metropolitan,6482.0
991,Box Hill,2/4 Collins St,3,u,775000.0,S,Fletchers,17/09/2016,13.1,3128.0,...,1.0,1.0,211.0,108.0,1980.0,Whitehorse,-37.8275,145.1232,Eastern Metropolitan,4605.0
4401,Northcote,16/116 Arthurton Rd,2,u,418000.0,SP,hockingstuart,7/05/2016,5.5,3070.0,...,1.0,1.0,1882.0,NaN,NaN,Darebin,-37.7680,144.9895,Northern Metropolitan,11364.0
7450,Altona North,269 Blackshaws Rd,3,h,916000.0,SP,Hunter,20/05/2017,11.1,3025.0,...,1.0,2.0,606.0,109.0,1970.0,Hobsons Bay,-37.8326,144.8594,Western Metropolitan,5132.0
3257,Heidelberg Heights,13 Altona St,4,h,856000.0,S,Miles,22/08/2016,10.5,3081.0,...,1.0,2.0,722.0,208.0,1975.0,Banyule,-37.7477,145.0565,Eastern Metropolitan,2947.0
1400,Brunswick West,64 Pearson St,3,h,955000.0,S,hockingstuart,7/05/2016,5.9,3055.0,...,1.0,0.0,220.0,93.0,1930.0,Moreland,-37.7659,144.9494,Northern Metropolitan,7082.0


In [6]:
# 필요한 데이터만 사용하기로 합니다.

df = mel_price[['Regionname', 'Type', 'Rooms', 'Bathroom', 'Landsize', 'Price']]

In [7]:
df.head()

,Regionname,Type,Rooms,Bathroom,Landsize,Price
0,Northern Metropolitan,h,2,1.0,202.0,1480000.0
1,Northern Metropolitan,h,2,1.0,156.0,1035000.0
2,Northern Metropolitan,h,3,2.0,134.0,1465000.0
3,Northern Metropolitan,h,3,2.0,94.0,850000.0
4,Northern Metropolitan,h,4,1.0,120.0,1600000.0


In [8]:
df.nunique()

Regionname       8
Type             3
Rooms            9
Bathroom         9
Landsize      1448
Price         2204
dtype: int64

# 지역에 따른 집값 차이가 유의미한가?

In [9]:
df['Regionname'].unique()

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

In [12]:
# Metropolitan 과 Victoria 로 나누어본다.

df[['Direction', 'Region']] = df['Regionname'].str.split(' ', n = 2, expand=True)
df.head()

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_17921/1653118217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Direction', 'Region']] = df['Regionname'].str.split(' ', n = 2, expand=True)
/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_17921/1653118217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Direction', 'Region']] = df['Regionname'].str.split(' ', n = 2, expand=True)


,Regionname,Type,Rooms,Bathroom,Landsize,Price,Direction,Region
0,Northern Metropolitan,h,2,1.0,202.0,1480000.0,Northern,Metropolitan
1,Northern Metropolitan,h,2,1.0,156.0,1035000.0,Northern,Metropolitan
2,Northern Metropolitan,h,3,2.0,134.0,1465000.0,Northern,Metropolitan
3,Northern Metropolitan,h,3,2.0,94.0,850000.0,Northern,Metropolitan
4,Northern Metropolitan,h,4,1.0,120.0,1600000.0,Northern,Metropolitan


In [13]:
df['Region'].value_counts()

Metropolitan    13454
Victoria          126
Name: Region, dtype: int64

In [14]:
df[df['Region'] == 'Metropolitan']['Price'].mean()

1080242.4607551657

In [15]:
df[df['Region'] == 'Victoria']['Price'].mean()

588950.253968254

차이가 꽤 존재한다. 거의 두 배 차이나는데 데이터 분석가로서, 다른 요인들을 먼저 분석해보려 한다.

In [16]:
df['Type'].unique()

array(['h', 'u', 't'], dtype=object)

In [20]:
df['Type'].value_counts()

h    9449
u    3017
t    1114
Name: Type, dtype: int64

In [21]:
df[df['Type'] == 'h']['Price'].mean()

1242664.761138745

In [22]:
df[df['Type'] == 'u']['Price'].mean()

605127.4845873384

In [23]:
df[df['Type'] == 't']['Price'].mean()

933735.0538599641

Metropolitan  집값이 훨씬 비싸고, 하우스 타입이 h가 비싼 편인 것으로 나타났다. 

Metropolitan 에 h 유형이 많은가?

In [27]:
df_h = df[df['Type'] =='h'][['Region', 'Price']]

In [30]:
df_h['Region'].value_counts()

Metropolitan    9326
Victoria         123
Name: Region, dtype: int64

재미있는건 h타입에 Victoria 지역의 집이 많다.  

126개 중 123개가 있다고? 그렇다면 각각의 평균 가격은?

In [33]:
pd.crosstab(index = df['Region'], columns = df['Type'], values = df['Price'], aggfunc = 'mean' )

Type,h,t,u
Region,,,
Metropolitan,1.251246e+06,933735.05386,605270.610816
Victoria,5.920629e+05,NaN,461333.333333


In [34]:
pd.options.display.float_format = '{:.2f}'.format
pd.crosstab(index = df['Region'], columns = df['Type'], values = df['Price'], aggfunc = 'mean' )

Type,h,t,u
Region,,,
Metropolitan,1251245.51,933735.05,605270.61
Victoria,592062.86,NaN,461333.33
